In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv('BIT_2023_1차.csv')
data.head(3)

,time,Open,High,Low,Close,Volume,returns,volatility,price,quote_qty,is_buyer_maker,quote_qty_cal
0,2023-01-01 0:00,16537.5,16540.9,16504.0,16527.0,5381.399,NaN,NaN,16524.52574,7264.129209,16494,88925066.30
1,2023-01-01 1:00,16527.1,16554.3,16524.1,16550.4,3210.826,0.001416,NaN,16537.21599,6819.889969,8705,53098123.05
2,2023-01-01 2:00,16550.5,16557.1,16534.8,16542.4,2399.668,-0.000483,NaN,16545.81814,6030.420093,8468,39704470.32


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   time            8760 non-null   object 
 1   Open            8736 non-null   float64
 2   High            8736 non-null   float64
 3   Low             8736 non-null   float64
 4   Close           8736 non-null   float64
 5   Volume          8760 non-null   float64
 6   returns         8748 non-null   float64
 7   volatility      8574 non-null   float64
 8   price           8736 non-null   float64
 9   quote_qty       8736 non-null   float64
 10  is_buyer_maker  8760 non-null   int64  
 11  quote_qty_cal   8736 non-null   float64
dtypes: float64(10), int64(1), object(1)
memory usage: 821.4+ KB


In [4]:
data.isna().sum()

time                0
Open               24
High               24
Low                24
Close              24
Volume              0
returns            12
volatility        186
price              24
quote_qty          24
is_buyer_maker      0
quote_qty_cal      24
dtype: int64

In [5]:
data.set_index('time', inplace=True)
data.index=pd.to_datetime(data.index)

In [6]:
# 결측치 처리: 선형 보간
data_interpolated = data.interpolate()

# 결측치 처리: 평균값 대체
data_mean_filled = data.fillna(data.mean())

# 정확도 계산
accuracy_interpolated = (data['volatility'].notna() == data_interpolated['volatility'].notna()).mean()

accuracy_mean_filled = (data['volatility'].notna() == data_mean_filled['volatility'].notna()).mean()

# 결과 출력
print("선형 보간 결과:")
print(data_interpolated)
print("\n평균값 대체 결과:")
print(data_mean_filled)

print("\n선형 보간의 정확도:", accuracy_interpolated)
print("\n평균값 대체의 정확도:", accuracy_mean_filled)

선형 보간 결과:
                        Open     High      Low    Close     Volume   returns  \
time                                                                           
2023-01-01 00:00:00  16537.5  16540.9  16504.0  16527.0   5381.399       NaN   
2023-01-01 01:00:00  16527.1  16554.3  16524.1  16550.4   3210.826  0.001416   
2023-01-01 02:00:00  16550.5  16557.1  16534.8  16542.4   2399.668 -0.000483   
2023-01-01 03:00:00  16542.5  16542.5  16515.0  16529.3   3214.480 -0.000792   
2023-01-01 04:00:00  16529.2  16530.4  16508.8  16517.8   3150.954 -0.000696   
...                      ...      ...      ...      ...        ...       ...   
2023-12-31 19:00:00  42701.7  42741.9  42624.7  42659.9   3944.096 -0.000977   
2023-12-31 20:00:00  42659.9  42724.5  42543.3  42599.1   4730.936 -0.001425   
2023-12-31 21:00:00  42599.2  42717.0  42558.2  42558.9   3794.010 -0.000944   
2023-12-31 22:00:00  42559.0  42629.5  42111.9  42294.8  11952.346 -0.006206   
2023-12-31 23:00:00  42294.8  

정확도는 실제 값과 예측 값이 얼마나 일치하는지를 측정하는 지표. 

1. `data['volatility'].notna()`는 'volatility' 열에서 결측치가 아닌 값들을 True로, 결측치를 False로 표시한 시리즈를 생성
2. `data_interpolated['volatility'].notna()`는 보간된 데이터프레임에서 'volatility' 열에서 결측치가 아닌 값들을 True로, 결측치를 False로 표시한 시리즈를 생성
3. `(data['volatility'].notna() == data_interpolated['volatility'].notna())`는 두 시리즈를 비교하여 같은 위치의 값이 일치하는 경우 True, 일치하지 않는 경우 False로 표시한 시리즈를 생성
4. `.mean()`은 True의 비율을 계산하여 정확도를 구한다. True의 비율은 총 True 개수를 전체 데이터 개수로 나눈 값

> 위의 코드에서 `accuracy_interpolated`는 결측치를 보간한 후의 'Open' 열과 원래 데이터의 'Open' 열이 일치하는 비율을 나타내는 정확도이다. 0부터 1 사이의 값을 가지며, 1에 가까울수록 정확도가 높다고 할 수 있다.

In [7]:
# 결측치 처리: 중간값 대체
df_median_filled = data.fillna(data.median())

# 결측치 처리: 최빈값 대체
df_mode_filled = data.apply(lambda x: x.fillna(x.value_counts().index[0]))

accuracy_median_filled = (data['volatility'].notna() == df_median_filled['volatility'].notna()).mean()
accuracy_mode_filled = (data['volatility'].notna() == df_mode_filled['volatility'].notna()).mean()

# 결과 출력
print("\n중간값 대체 결과:")
print(df_median_filled)
print("\n최빈값 대체 결과:")
print(df_mode_filled)

print("\n중간값 대체의 정확도:", accuracy_median_filled)
print("\n최빈값 대체의 정확도:", accuracy_mode_filled)


중간값 대체 결과:
                        Open     High      Low    Close     Volume   returns  \
time                                                                           
2023-01-01 00:00:00  16537.5  16540.9  16504.0  16527.0   5381.399  0.000071   
2023-01-01 01:00:00  16527.1  16554.3  16524.1  16550.4   3210.826  0.001416   
2023-01-01 02:00:00  16550.5  16557.1  16534.8  16542.4   2399.668 -0.000483   
2023-01-01 03:00:00  16542.5  16542.5  16515.0  16529.3   3214.480 -0.000792   
2023-01-01 04:00:00  16529.2  16530.4  16508.8  16517.8   3150.954 -0.000696   
...                      ...      ...      ...      ...        ...       ...   
2023-12-31 19:00:00  42701.7  42741.9  42624.7  42659.9   3944.096 -0.000977   
2023-12-31 20:00:00  42659.9  42724.5  42543.3  42599.1   4730.936 -0.001425   
2023-12-31 21:00:00  42599.2  42717.0  42558.2  42558.9   3794.010 -0.000944   
2023-12-31 22:00:00  42559.0  42629.5  42111.9  42294.8  11952.346 -0.006206   
2023-12-31 23:00:00  42294.8

In [8]:
# 0으로 결측치 대체
df_zero_filled = data.fillna(0)

accuracy_zero_filled = (data['volatility'].notna() == df_zero_filled['volatility'].notna()).mean()

# 결과 출력
print("0으로 대체한 결과:")
print(df_zero_filled)
print("\n0으로 대체의 정확도:", accuracy_zero_filled)

0으로 대체한 결과:
                        Open     High      Low    Close     Volume   returns  \
time                                                                           
2023-01-01 00:00:00  16537.5  16540.9  16504.0  16527.0   5381.399  0.000000   
2023-01-01 01:00:00  16527.1  16554.3  16524.1  16550.4   3210.826  0.001416   
2023-01-01 02:00:00  16550.5  16557.1  16534.8  16542.4   2399.668 -0.000483   
2023-01-01 03:00:00  16542.5  16542.5  16515.0  16529.3   3214.480 -0.000792   
2023-01-01 04:00:00  16529.2  16530.4  16508.8  16517.8   3150.954 -0.000696   
...                      ...      ...      ...      ...        ...       ...   
2023-12-31 19:00:00  42701.7  42741.9  42624.7  42659.9   3944.096 -0.000977   
2023-12-31 20:00:00  42659.9  42724.5  42543.3  42599.1   4730.936 -0.001425   
2023-12-31 21:00:00  42599.2  42717.0  42558.2  42558.9   3794.010 -0.000944   
2023-12-31 22:00:00  42559.0  42629.5  42111.9  42294.8  11952.346 -0.006206   
2023-12-31 23:00:00  42294.8

In [9]:
data_spline_interpolated = data.interpolate(method='spline', order=1)

accuracy_spline_interpolated = (data['volatility'].notna() == data_spline_interpolated['volatility'].notna()).mean()

# 결과 출력
print("스플라인 보간법을 적용한 결과:")
print(data_spline_interpolated)
print("\n스플라인 보간법의 정확도:", accuracy_spline_interpolated)

스플라인 보간법을 적용한 결과:
                        Open     High      Low    Close     Volume   returns  \
time                                                                           
2023-01-01 00:00:00  16537.5  16540.9  16504.0  16527.0   5381.399       NaN   
2023-01-01 01:00:00  16527.1  16554.3  16524.1  16550.4   3210.826  0.001416   
2023-01-01 02:00:00  16550.5  16557.1  16534.8  16542.4   2399.668 -0.000483   
2023-01-01 03:00:00  16542.5  16542.5  16515.0  16529.3   3214.480 -0.000792   
2023-01-01 04:00:00  16529.2  16530.4  16508.8  16517.8   3150.954 -0.000696   
...                      ...      ...      ...      ...        ...       ...   
2023-12-31 19:00:00  42701.7  42741.9  42624.7  42659.9   3944.096 -0.000977   
2023-12-31 20:00:00  42659.9  42724.5  42543.3  42599.1   4730.936 -0.001425   
2023-12-31 21:00:00  42599.2  42717.0  42558.2  42558.9   3794.010 -0.000944   
2023-12-31 22:00:00  42559.0  42629.5  42111.9  42294.8  11952.346 -0.006206   
2023-12-31 23:00:00  4

In [16]:
import statsmodels.api as sm

def interpolate_arima(data, order):
    # 결측치 보간 전 ARIMA 모델 훈련
    model = sm.tsa.ARIMA(data.dropna(), order=order)
    results = model.fit()

    # 보간할 결측치 선택
    missing_indices = data[data.isnull()].index

    # 결측치 보간 및 정확도 계산
    actual_values = []
    predicted_values = []
    for idx in missing_indices:
        # ARIMA 모델을 사용하여 결측치 예측
        predicted_value = results.get_forecast(steps=1).predicted_mean.iloc[0]

        # 결측치 보간
        data[idx] = predicted_value

        # 실제 값과 예측 값 저장
        actual_values.append(data[idx])
        predicted_values.append(predicted_value)

    return data

# volatility 컬럼에 대해 ARIMA 모델을 적용하여 결측치 보간 및 정확도 계산
arima_data = interpolate_arima(data['volatility'], (1, 0, 2))

# 정확도 계산
accuracy_arima_spline = (data['volatility'].notna() == data_spline_interpolated['volatility'].notna()).mean()

print("arima을 적용한 결과:")
print(arima_data)
print("\arima의 정확도:", accuracy_arima_spline)

C:\Users\jwoo\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)
C:\Users\jwoo\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)
C:\Users\jwoo\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)
C:\Users\jwoo\anaconda3\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


arima을 적용한 결과:
time
2023-01-01 00:00:00    0.003006
2023-01-01 01:00:00    0.003006
2023-01-01 02:00:00    0.003006
2023-01-01 03:00:00    0.003006
2023-01-01 04:00:00    0.003006
                         ...   
2023-12-31 19:00:00    0.003040
2023-12-31 20:00:00    0.002863
2023-12-31 21:00:00    0.002673
2023-12-31 22:00:00    0.003008
2023-12-31 23:00:00    0.002975
Name: volatility, Length: 8760, dtype: float64
rima의 정확도: 0.997716894977169
